In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import queue
import threading
import re
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer
import sys
from sksurgerynditracker.nditracker import NDITracker
import os
import random
import multiprocessing


In [2]:
# Tool Tip Offset (Medtronic 960-556) is from Mike's GitHub Repo
TOOL_TIP_OFFSET = np.array([-304.5728,-0.3053,-0.1412, 1])

In [3]:
# Initialize the NDI tracker
SETTINGS = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,
    # Mac Path
    # "romfiles": ["/Users/yizheng/Desktop/Dartmouth/lab work/ARPAH/Polaris Vega XT/tool_defs/medtronic_chicken_foot_960_556.rom"]
    # Windows Path
    "romfiles": ["C:\\Users\\f007wsq\\Desktop\\tools_and_models\\tool_defs\\medtronic_chicken_foot_960_556.rom"]
}
tracker = NDITracker(SETTINGS)
tracker.start_tracking()

In [4]:
# Initialize vtk
tool_reader = vtk.vtkSTLReader() # read stl file
tool_reader.SetFileName("C:\\Users\\f007wsq\\Desktop\\tools_and_models\\tool_stl_models\\medtronic_chicken_foot.STL") # set the file name
tool_reader.Update() # update the reader

tool_mapper = vtk.vtkPolyDataMapper() # create a mapper, a mapper is used to map the data to the graphics primitives
tool_mapper.SetInputData(tool_reader.GetOutput()) # set the input to the reader

tool_actor = vtk.vtkActor() # create an actor, an actor is used to display the data
tool_actor.SetMapper(tool_mapper) # set the mapper to the actor

renderer = vtk.vtkRenderer() # create a renderer, a renderer is used to display the data
render_window = vtk.vtkRenderWindow() # create a render window, a render window is used to display the data
render_window.AddRenderer(renderer) # add the renderer to the render window
render_window.SetSize(1000, 800) # set the size of the render window

render_window_interactor = vtk.vtkRenderWindowInteractor() # create a render window interactor, a render window interactor is used to interact with the data
render_window_interactor.SetRenderWindow(render_window) # set the render window to the render window interactor

renderer.AddActor(tool_actor) # add the actor to the renderer
renderer.SetBackground(1, 1, 1) # set the background of the renderer

# 创建坐标轴
axes = vtk.vtkAxesActor()
axes.SetTotalLength(200, 200, 200)  # 设置坐标轴的长度
axes.SetAxisLabels(True)

# 设置坐标轴标签的位置
axes.GetXAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetYAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetZAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()

# 设置坐标轴标签
axes.GetXAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(1, 0, 0)  # X轴为红色
axes.GetYAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 1, 0)  # Y轴为绿色
axes.GetZAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 0, 1)  # Z轴为蓝色

# 将坐标轴添加到 renderer
renderer.AddActor(axes)

# 设定摄像机视角，固定在 (0, 0, -3000) 位置
renderer.GetActiveCamera().SetPosition(-1000, 1000, -5000)  # 摄像机位置
renderer.GetActiveCamera().SetFocalPoint(0, 0, -2000)  # 摄像机焦点
renderer.GetActiveCamera().SetViewUp(-1, 0, 0)  # 设置“上”方向
renderer.GetActiveCamera().SetClippingRange(1, 3000)  # 设置裁剪范围

In [ ]:
cwd = os.getcwd()
models = []

# For Mac Path
# for file in os.listdir(cwd + '/' + 'slicer_files/'):

# For Windows Path
for file in os.listdir("C:\\Users\\f007wsq\Desktop\\tools_and_models\\slicer_files"):
    if file.endswith('.vtk') or file.endswith('.stl'):
        models.append(file)

print(models)

In [6]:
# transformation_matrix is calculated using "phantom_registration.ipynb"
transformation_matrix = np.array([[-1.78752390e-02,  9.97490742e-01,  6.85032455e-02,  1.20512994e+02],
 [ 9.99681492e-01,  1.90512108e-02, -1.65519287e-02,  5.60709263e+01],
 [-1.78154654e-02,  6.81855570e-02, -9.97513578e-01, -1.31217738e+03],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [7]:
# Loop through model files, load them, and add them to the renderer
for file in models:
    if file.endswith(".stl"):
        model_reader = vtk.vtkSTLReader()

        # Mac Path
        # reader.SetFileName(cwd + '/' + 'slicer_files/' + file)

        # Windows Path
        model_reader.SetFileName("C:\\Users\\f007wsq\\Desktop\\tools_and_models\\slicer_files\\" + file)

    elif file.endswith(".vtk"):
        model_reader = vtk.vtkPolyDataReader()
        
        # Mac Path
        # reader.SetFileName(cwd + '/' + 'slicer_files/' + file)

        # Windows Path
        model_reader.SetFileName("C:\\Users\\f007wsq\\Desktop\\tools_and_models\\slicer_files\\" + file)
    else:
        continue  # Skip unsupported formats

    # Update the reader to load the data
    model_reader.Update()

    # Create a mapper and actor to display the model
    model_mapper = vtk.vtkPolyDataMapper()
    model_mapper.SetInputData(model_reader.GetOutput())
    model_actor = vtk.vtkActor()
    model_actor.SetMapper(model_mapper)
    model_actor.GetProperty().SetColor(random.randint(0, 255)/255, random.randint(0, 255)/255, random.randint(0, 255)/255)

    transform = vtk.vtkTransform()
    transform.SetMatrix(transformation_matrix.flatten()) 
    model_actor.SetUserTransform(transform)


    if file.startswith("Box"):
        model_actor.GetProperty().SetOpacity(0.05)  # 设置透明度 (0.0 - 完全透明, 1.0 - 完全不透明)
    if file.startswith("Box_1"):
        model_actor.GetProperty().SetOpacity(0.00)  # 设置透明度 (0.0 - 完全透明, 1.0 - 完全不透明)
    # Add the actor to the renderer
    renderer.AddActor(model_actor)

In [8]:
def get_tracking_data(tracker):
    """Fetch latest tracking data"""
    port_handles, timestamps, framenumbers, transformation, quality = tracker.get_frame()
    transformation = np.array(transformation[0])
    return timestamps, transformation, quality

In [9]:
data_lock = threading.Lock()
data_ready = threading.Event()
data_queue = queue.Queue()

def polaris_thread():
    global polaris_matrix
    while True:
        _, new_matrix, _ = get_tracking_data(tracker)
        data_queue.put(new_matrix)
        time.sleep(1/400)

In [10]:
# VTK Update Thread
def update_pose(obj, event):
    global polaris_matrix

    if not data_queue.empty():
        polaris_matrix = data_queue.get()
        transform = vtk.vtkTransform()
        transform.SetMatrix(polaris_matrix.flatten()) 
        tool_actor.SetUserTransform(transform)
        render_window.Render()
        print("pose updated")

In [ ]:
threading.Thread(target=polaris_thread, daemon=False).start()

render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.AddObserver("TimerEvent", update_pose)
render_window_interactor.CreateRepeatingTimer(1)
render_window_interactor.Start()